In [1]:
!pip install simpletransformers pandas scikit-learn torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
# Load dataset
df = pd.read_csv("/content/filtered_sentiment_dataset.csv")

In [4]:
# Map sentiment labels to numerical values
label_map = {"Negative": 0, "Neutral": 1, "Positive": 2}
reverse_label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
df["sentiment"] = df["sentiment"].map(label_map)

In [5]:
# Drop NaN values
df.dropna(inplace=True)

In [6]:
# Split dataset into train (80%), validation (10%), and test (10%)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["sentiment"])
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["sentiment"])

In [7]:
# Rename columns to match simpletransformers expectations
train_df = train_df.rename(columns={"text": "text", "sentiment": "labels"})
valid_df = valid_df.rename(columns={"text": "text", "sentiment": "labels"})
test_df = test_df.rename(columns={"text": "text", "sentiment": "labels"})

In [8]:
# Convert labels to integer type
train_df["labels"] = train_df["labels"].astype(int)
valid_df["labels"] = valid_df["labels"].astype(int)
test_df["labels"] = test_df["labels"].astype(int)


In [9]:
# Define model arguments
model_args = ClassificationArgs()
model_args.num_train_epochs = 7
model_args.train_batch_size = 16
model_args.eval_batch_size = 16
model_args.learning_rate = 5e-6
model_args.weight_decay = 0.01
model_args.overwrite_output_dir = True
model_args.save_best_model = True
model_args.evaluate_during_training = True
model_args.use_early_stopping = True
model_args.early_stopping_patience = 2
model_args.output_dir = "muril-sentiment-model"
model_args.best_model_dir = "muril-best-model"

In [10]:
# Initialize the MuRIL model
model = ClassificationModel(
    "bert", "google/muril-base-cased",
    num_labels=3,
    args=model_args,
    use_cuda=torch.cuda.is_available()
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/953M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [11]:
# Train the model
model.train_model(train_df, eval_df=valid_df)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 7:   0%|          | 0/630 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


...

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


(4410,
 defaultdict(list,
             {'global_step': [630,
               1260,
               1890,
               2000,
               2520,
               3150,
               3780,
               4000,
               4410],
              'train_loss': [0.5902099609375,
               0.364105224609375,
               0.6353657841682434,
               0.4076690673828125,
               0.5882466435432434,
               0.466461181640625,
               0.448486328125,
               0.5507392883300781,
               0.2102152556180954],
              'mcc': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
              'eval_loss': [0.6549929848200158,
               0.49705183468287506,
               0.47252323469029195,
               0.4703284154964399,
               0.4416508089892472,
               0.43336416007597234,
               0.4313452998294106,
               0.4306125561647777,
               0.42935838993591596]}))

In [12]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(valid_df, acc=accuracy_score)
print(f"Model Accuracy: {result['acc']:.4f}")


  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Model Accuracy: 0.8381


In [13]:
# Function to predict sentiment
def predict_sentiment(text, model):
    predictions, raw_outputs = model.predict([text])
    confidence = torch.softmax(torch.tensor(raw_outputs[0]), dim=0).tolist()
    sentiment = reverse_label_map[predictions[0]]
    print(f"Text: {text}\nPredicted Sentiment: {sentiment}\nConfidence Scores: {confidence}\n")
    return sentiment

In [14]:
# Predict on test set
for text in test_df["text"].tolist()[:10]:
    predict_sentiment(text, model)

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:2188: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Text:  Thala  a sami  I am wit
Predicted Sentiment: Positive
Confidence Scores: [0.10203440184017111, 0.012640818269233554, 0.8853247798905953]



0it [00:00, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Text:  Ivan comedian aatchay epde hero Anaaan comedy hero subject tan sari Sivakarthigeyn kku
Predicted Sentiment: Positive
Confidence Scores: [0.30087164455201576, 0.10519449194625981, 0.5939338635017245]



0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Text:  Thalaivernna naama super Strdahan pinnittaru trailere keddha irukkunna padam vandha vera leval
Predicted Sentiment: Positive
Confidence Scores: [0.11043818963570312, 0.013550636680356356, 0.8760111736839404]



In [15]:
# Save the trained model
model.save_model("my_muril_sentiment_model")